In [1]:
import numpy as np
from random import random

In [2]:
class MLP(object):
    def __init__(self ,num_inputs = 3, hidden_layers = [3, 3], num_outputs = 2):
        self.num_inputs = num_inputs
        self.hidden_layers = hidden_layers
        self.num_outputs = num_outputs
        
        #create a representation of the layers
        layers = [num_inputs] + hidden_layers + [num_outputs]
        
        #create random connection weights for the layers 
        weights = []
        for i in range(len(layers) - 1):
            w = np.random.rand(layers[i], layers[i + 1])
            weights.append(w)
        self.weights = weights
        
        #save derivates per layer
        derivatives = []
        for i in range(len(layers) - 1):
            d = np.zeros((layers[i], layers[i+1]))
            derivatives.append(d)
        self.derivatives = derivatives
        
        #save activations per layer
        activations = []
        
        for i in range(len(layers)):
            a = np.zeros(layers[i])
            activations.append(a)
        self.activations = activations
        
    def forward_propagte(self,inputs):
        #input layer activation is just the input itself
        activations = inputs 
        
        #save the activations for backpropogations
        self.activations[0] = activations
        
        #iter through the network layers
        for i,w in enumerate(self.weights):
            #calculate matrix multiplications btw previous activations and weight matrixs
            net_inputs = np.dot(activations, w)
            
            #apply sigmoid activation function
            activations = self.sigmoid(net_inputs)
            
            #save the activations for backpropogation
            self.activations[i + 1] = activations
        #return output layer activation 
        return activations
    
    def back_propagate(self, error):
        
        #iter in backward in network layers
        for i in reversed(range(len(self.derivatives))):
            
            #get activations for previous layer
            activations = self.activations[i + 1]
            
            #apply sigmoid derivative function
            delta = error*self.sigmoid_derivate(activations)
            
            #reshape fuction as a 2d array
            delta_re = delta.reshape(delta.shape[0], -1).T
            
            #get activation for current layer
            current_activations = self.activations[i]  # --> [1,3,4,5] --> [[1],[2],[3],[4]]
            
            #reshape activation as 2d col matrix
            current_activations = current_activations.reshape(current_activations.shape[0], -1)
            
            #save derivate after applying matrix multiplications
            self.derivatives[i] = np.dot(current_activations,delta_re)
            
            #backpropagate the next error
            error = np.dot(delta, self.weights[i].T)
    
    def sigmoid(self, x):
        y = 1.0/(1 + np.exp(-x))
        return y
    
    def sigmoid_derivate(self, x):
        return x*(1.0 - x)
    
    def mse(self, target,output):
        return np.average((target - output) ** 2)
    
    def gradient_descent(self, learningRate = 1):
        #update the weight by stepping down the gradient
        for i in range(len(self.weights)):
            weights = self.weights[i]
            derivatives =self.derivatives[i]
            weights += derivatives * learningRate
    def train(self, inputs, targets, epochs, learning_rate):
        
        #now enter the training data
        for i in range(epochs):
            sum_errors = 0
            
            #iter through all training data 
            for j, input in enumerate(inputs):
                target = targets[j]
                
                #activate the network!
                output = self.forward_propagte(input)
                
                error = target - output
                
                self.back_propagate(error)
                
                #perform gradient descent on derivate ##this will update weights
                self.gradient_descent(learning_rate)
                
                #keep track of MSE for report later
                sum_errors+= self.mse(target,output)
            #Epoch complete , report the training error
            print("ERROR: {} at each epoch {}".format(sum_errors/len(items), i+1))
        print("Training complete!")
        print("======")

In [3]:
items = np.array([[random()/2 for _ in range(2)] for _ in range(1000)])

In [4]:
targets = np.array([[i[0] + i[1]] for i in items])

In [5]:
#create mlp with one hidden layer 
mlp = MLP(2, [5], 1)

In [6]:
mlp.train(items, targets, 50 , 0.1)

ERROR: 0.04901593958420339 at each epoch 1
ERROR: 0.039756727759117726 at each epoch 2
ERROR: 0.039602475424942066 at each epoch 3
ERROR: 0.03943845605968112 at each epoch 4
ERROR: 0.03925286318155021 at each epoch 5
ERROR: 0.03903282766451631 at each epoch 6
ERROR: 0.03876365703325006 at each epoch 7
ERROR: 0.038428126256752106 at each epoch 8
ERROR: 0.03800582181892707 at each epoch 9
ERROR: 0.03747263025814831 at each epoch 10
ERROR: 0.036800557651099186 at each epoch 11
ERROR: 0.03595820862324541 at each epoch 12
ERROR: 0.03491243125335593 at each epoch 13
ERROR: 0.033631770614514375 at each epoch 14
ERROR: 0.03209226473562041 at each epoch 15
ERROR: 0.03028542899801859 at each epoch 16
ERROR: 0.02822678150555841 at each epoch 17
ERROR: 0.025961424911857864 at each epoch 18
ERROR: 0.023562587719612792 at each epoch 19
ERROR: 0.021121338257783506 at each epoch 20
ERROR: 0.018730354008770822 at each epoch 21
ERROR: 0.01646814300722741 at each epoch 22
ERROR: 0.014389406005824642 at e

In [7]:
# create dummy data 
input = np.array([0.3, 0.1])
target = np.array([0.4])

In [8]:
#get a prediction
output = mlp.forward_propagte(input)

In [9]:
print("Our network believes that {} + {} is equal to {}".format(input[0], input[1], output[0]))

Our network believes that 0.3 + 0.1 is equal to 0.41078154178626136
